In [ ]:
#Utilitarios
from statsmodels.stats.descriptivestats import Description
from sklearn.model_selection import train_test_split
from mne.externals.pymatreader import read_mat
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import FastICA
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import os

#Rede Neural
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense, Conv2D
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Dropout, BatchNormalization 
import keras_tuner as kt
from tensorflow import keras
os.environ['TENSORBOARD_BINARY'] = '/path/to/envs/my_env/bin/tensorboard'
%load_ext tensorboard

In [ ]:
subject1_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw01.mat')
subject1_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw02.mat')
subject1_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject1_mat\train_subject1_raw03.mat')

subject2_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw01.mat')
subject2_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw02.mat')
subject2_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject2_mat\train_subject2_raw03.mat')

subject3_01 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw01.mat')
subject3_02 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw02.mat')
subject3_03 = read_mat(r'C:\Users\Alex\Desktop\EEG\subject3_mat\train_subject3_raw03.mat')

In [ ]:
#subject1_01['nfo']              #items()

In [ ]:
subject_X = np.concatenate( (subject1_01['X'], subject1_02['X'], subject1_03['X'], subject2_01['X'], subject2_02['X'], subject2_03['X'], subject3_01['X'], subject3_02['X'], subject3_03['X'] ), axis = 0)

In [ ]:
subject_Y = np.concatenate( (subject1_01['Y'], subject1_02['Y'], subject1_03['Y'], subject2_01['Y'], subject2_02['Y'], subject2_03['Y'], subject3_01['Y'], subject3_02['Y'], subject3_03['Y'] ), axis = 0)
subject_Y.shape

In [ ]:
# Pre-processamento dos dados
ICA = FastICA()

subject_X = ICA.fit_transform(subject_X)

In [ ]:
Y = np.take( subject_Y , np.arange(0, len( subject_X ), 512) , axis=None) 

In [ ]:
X = np.asarray(  np.split( np.transpose( subject_X ), 2141, axis = 1 ))
X.shape

In [ ]:
t = pd.DataFrame( Description( X[0].flatten() ).summary() ).T # Vai duplica a primeira linha

for i in range(  X.shape[0] ):
    r = pd.DataFrame( Description( X[i].flatten() ).summary() ).T
    t = t.append(r)
    

In [ ]:
z = np.empty( 16384 )
for j in range(  X.shape[0] ):
    v = X[j].flatten() 
    z = np.vstack((z,v))
z = np.delete(arr = z, obj = 0, axis = 0 )

In [ ]:
#z = np.delete(arr = z, obj = 0, axis = 0 )

In [ ]:
pca = PCA( n_components = 8) # mle - 12
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print(pca.singular_values_)

In [ ]:
t.to_csv(r'C:\Users\Alex\Desktop\testao.csv', decimal = ',', sep = '+')

In [ ]:
X = pd.read_excel(r'C:\Users\Alex\Desktop\x_eeg_svm.xlsx')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( z, Y, test_size=0.20, random_state=42, stratify = Y) 

NUM_CLASSES = 3

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
#y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

#Faz One-Hot Encoding
y_train = to_categorical(y_train, NUM_CLASSES)
#y_val = to_categorical(y_val, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [ ]:
from sklearn.svm import NuSVC
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
import sklearn as sk

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# SVM

In [ ]:
tuned_parameters2 = [{'SVC__kernel': ['rbf'], 'SVC__gamma': [1000, 100, 10, 1,0.1,0.01,0.001, 'scale'],  'SVC__C': [0.01,0.001, 0.1,1, 10, 100, 1000], 'SVC__decision_function_shape' : ['ovo', 'ovr'], 'SVC__class_weight':['balanced', None]},
                    {'SVC__kernel': ['poly'], 'SVC__gamma': [1000, 100, 10, 1,0.1,0.01,0.001, 'scale'], 'SVC__C': [0.01,0.001, 0.1,1, 10, 100, 1000], 'SVC__decision_function_shape' : ['ovo', 'ovr'], 'SVC__class_weight':['balanced', None], 'SVC__degree': [2,3,4]},
                    {'SVC__kernel': ['sigmoid'], 'SVC__gamma': [1000, 100, 10, 1,0.1,0.01,0.001, 'scale'], 'SVC__C': [0.01,0.001, 0.1,1, 10, 100, 1000], 'SVC__decision_function_shape' : ['ovo', 'ovr'], 'SVC__class_weight':['balanced', None]},
                    {'SVC__kernel': ['linear'], 'SVC__C': [0.01,0.001, 0.1,1, 10, 100, 1000], 'SVC__decision_function_shape' : ['ovo', 'ovr'], 'SVC__class_weight':['balanced', None]}
                    ]  

In [ ]:
#tuned_parameters2 = [{'SVC__kernel': ['rbf'], 'SVC__gamma': [1000, 100, 10, 1,0.1,0.01,0.001, 'scale'],  'SVC__C': [0.01,0.001, 0.1,1, 10, 100, 1000], 'SVC__decision_function_shape' : ['ovo', 'ovr'], 'SVC__class_weight':['balanced', None]}] 
tuned_parameters2 = [{'SVC__kernel': ['rbf'], 'SVC__gamma': [ 'scale'],}]  

In [ ]:
def confusion_matrix_scorer(clf, X, y):
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    return cm[0, 0]

In [ ]:
#scoring = ['accuracy', 'precision'] #str
#scoring = {'accuracy': make_scorer(accuracy_score),
#           'precision': make_scorer(precision_score, average = 'weighted'),
#           'recall': make_scorer(recall_score, average = 'weighted'),
#           'f1_macro': make_scorer(f1_score, average = 'macro'),
#            'f1_weighted': make_scorer(f1_score, average = 'weighted')} # Se average = None, é calculado a metrica por rotulo
scoring = {'score': confusion_matrix_scorer } 
#scoring = {'accuracy': make_scorer(accuracy_score),
#           'precision': make_scorer(precision_score, average = None)}


#pipe2 = Pipeline(steps=[ ('StandardScaler', StandardScaler()), ('PCA', PCA( n_components = 12 )), ('SVC', SVC()) ])
#pipe2 = Pipeline(steps=[ ('StandardScaler', StandardScaler()), ('TSNE', TSNE()), ('SVC', SVC()) ])
pipe2 = Pipeline(steps=[ ('StandardScaler', StandardScaler()), ('SVC', SVC()) ])

clf2 = GridSearchCV( pipe2, tuned_parameters2, scoring=scoring ,verbose=100, cv = 1, refit = make_scorer(accuracy_score) )  #, n_jobs = -1 #, refit='AUC'

clf2.fit( x_train  , y_train)

In [ ]:
clf2.cv_results_

In [ ]:
clf2.best_estimator_

# Fim